In [1]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]
# !{sys.executable} -m pip install pip rasterstats 

In [2]:
import ee
# ee.Authenticate()

In [3]:
ee.Initialize()

In [4]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [5]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [6]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [7]:
# read esa land cover
esa_land_cover = ee.ImageCollection('ESA/WorldCover/v100').first()

In [8]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/geo_ref.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM2,COD,ADM2
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


# Compute indicator

In [9]:
cities_indicators_GRE_3_1 = pd.DataFrame() 

In [11]:
for i in range(0,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    
        
    #read open space
    openspace_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    openspace_geo = requests.get(openspace_path).json()
    openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)

    ## Make an image, with the same projection as WorldCover, out of the OSM ways in the FC.
    WCprojection = esa_land_cover.projection(); 
    RecSitesImg = openspace_geo_ee.style(
      color='gray',
    ).reproject(
          crs= WCprojection
        )
    # create image with two bands: BuiltupPixels and UrbanOpenPixels
    Builtup = esa_land_cover.updateMask(esa_land_cover.eq(50)).rename("BuiltupPixels")
    UrbanOpen = RecSitesImg.updateMask(esa_land_cover.eq(50)).select(1).rename("UrbanOpenPixels")
    comb = Builtup.addBands([UrbanOpen])
    
    # Function to translate pixel counts into area and percents
    def CountToArea(feat):
        feat = ee.Feature(feat)
        FeatArea = ee.Number(feat.area(1)).multiply(0.000001)
        UrbanOpenArea = ee.Number(feat.getNumber('UrbanOpenPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
        BuiltupArea = ee.Number(feat.getNumber('BuiltupPixels')).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
        OpenAreaPctofBuiltUpArea = ee.Number(UrbanOpenArea).divide(ee.Number(BuiltupArea))

        return feat.set({
            #'TotalareaKM2': FeatArea,
            #'UrbanOpenAreaKM2': UrbanOpenArea,
            #'BuiltupAreaKM2': BuiltupArea,
            'OpenAreaPctofBuiltUpArea': OpenAreaPctofBuiltUpArea,
        })
    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    ## create FeatureCollection with pixels counts of Builtup and UrbanOpen for each feature
    OpenBuiltcount=comb.reduceRegions(
      reducer= ee.Reducer.count(), 
      collection= boundary_geo_ee, 
      scale= 10, 
      tileScale= 1
    )

    # apply CountToArea function to FeatureCollection
    OpenBuiltAreaPct = OpenBuiltcount.map(CountToArea).select(['geo_id','OpenAreaPctofBuiltUpArea'])

    # store in df and apend
    df = geemap.ee_to_pandas(OpenBuiltAreaPct)
    df = df.rename(columns={"OpenAreaPctofBuiltUpArea": "GRE_3_1_percentOpenSpaceinBuiltup"})
    cities_indicators_GRE_3_1 = cities_indicators_GRE_3_1.append(df)
    
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    ## create FeatureCollection with pixels counts of Builtup and UrbanOpen for each feature
    OpenBuiltcount=comb.reduceRegions(
      reducer= ee.Reducer.count(), 
      collection= boundary_geo_ee, 
      scale= 10, 
      tileScale= 1
    )
    
    # apply CountToArea function to FeatureCollection
    OpenBuiltAreaPct = OpenBuiltcount.map(CountToArea).select(['geo_id','OpenAreaPctofBuiltUpArea'])

    # store in df and apend
    df = geemap.ee_to_pandas(OpenBuiltAreaPct)
    df = df.rename(columns={"OpenAreaPctofBuiltUpArea": "GRE_3_1_percentOpenSpaceinBuiltup"})
    cities_indicators_GRE_3_1 = cities_indicators_GRE_3_1.append(df)

0

 geo_name: BRA-Salvador

 boundary_id_aoi: BRA-Salvador-ADM4union

 boundary_id_unit: BRA-Salvador-ADM4
1

 geo_name: COD-Bukavu

 boundary_id_aoi: COD-Bukavu-ADM3union

 boundary_id_unit: COD-Bukavu-ADM3
2

 geo_name: COD-Uvira

 boundary_id_aoi: COD-Uvira-ADM3union

 boundary_id_unit: COD-Uvira-ADM3
3

 geo_name: COG-Brazzaville

 boundary_id_aoi: COG-Brazzaville-ADM4union

 boundary_id_unit: COG-Brazzaville-ADM4
4

 geo_name: COL-Barranquilla

 boundary_id_aoi: COL-Barranquilla-ADM4union

 boundary_id_unit: COL-Barranquilla-ADM4
5

 geo_name: ETH-Addis_Ababa

 boundary_id_aoi: ETH-Addis_Ababa-ADM4union

 boundary_id_unit: ETH-Addis_Ababa-ADM4
6

 geo_name: ETH-Dire_Dawa

 boundary_id_aoi: ETH-Dire_Dawa-ADM3union

 boundary_id_unit: ETH-Dire_Dawa-ADM3
7

 geo_name: KEN-Nairobi

 boundary_id_aoi: KEN-Nairobi-ADM3union

 boundary_id_unit: KEN-Nairobi-ADM3
8

 geo_name: MDG-Antananarivo

 boundary_id_aoi: MDG-Antananarivo-ADM4union

 boundary_id_unit: MDG-Antananarivo-ADM4
9

 geo_na

Exception: Request payload size exceeds the limit: 10485760 bytes.

In [12]:
cities_indicators_GRE_3_1

,GRE_3_1_percentOpenSpaceinBuiltup,geo_id
0,0.043746,BRA-Salvador_ADM4-union_1
0,0.072329,BRA-Salvador_ADM4_1
1,0.103440,BRA-Salvador_ADM4_2
2,0.050375,BRA-Salvador_ADM4_3
3,0.048150,BRA-Salvador_ADM4_4
...,...,...
189,0.000000,MDG-Antananarivo_ADM4_190
190,0.000000,MDG-Antananarivo_ADM4_191
191,0.000000,MDG-Antananarivo_ADM4_192
192,0.000000,MDG-Antananarivo_ADM4_193


# Merge with indicator table

In [13]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_erictest.csv') 
cities_indicators#.head()

,geo_id,geo_level,geo_name,geo_parent_name
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador
1,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador
2,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador
3,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador
4,BRA-Salvador_ADM4_5,ADM4,Jardim Armação,BRA-Salvador
...,...,...,...,...
936,MEX-Monterrey_ADM2_15,ADM2,San Nicolás de los Garza,MEX-Monterrey
937,MEX-Monterrey_ADM2_16,ADM2,Hidalgo,MEX-Monterrey
938,MEX-Monterrey_ADM2_17,ADM2,Santa Catarina,MEX-Monterrey
939,MEX-Monterrey_ADM2_18,ADM2,Santiago,MEX-Monterrey


In [14]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace by new indicators")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [15]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = cities_indicators_GRE_3_1,
                                            indicator_name = "GRE_3_1_percentOpenSpaceinBuiltup")

add new indicators


In [16]:
cities_indicators_merged

,geo_id,geo_level,geo_name,geo_parent_name,GRE_3_1_percentOpenSpaceinBuiltup
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,0.072329
1,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,0.072329
2,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,0.103440
3,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,0.103440
4,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,0.050375
...,...,...,...,...,...
1781,MEX-Monterrey_ADM2_15,ADM2,San Nicolás de los Garza,MEX-Monterrey,NaN
1782,MEX-Monterrey_ADM2_16,ADM2,Hidalgo,MEX-Monterrey,NaN
1783,MEX-Monterrey_ADM2_17,ADM2,Santa Catarina,MEX-Monterrey,NaN
1784,MEX-Monterrey_ADM2_18,ADM2,Santiago,MEX-Monterrey,NaN


# Upload in aws s3

In [17]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [18]:
# upload to aws
key_data = 'data/indicators/cities_indicators_erictest.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [19]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')